In [1]:
import pandas as pd
import math
from typing import List
from datetime import datetime
from sqlalchemy import create_engine
from sqlalchemy import text

In [2]:
engine = create_engine('sqlite:///pare.db')

In [3]:
test = False

In [4]:
import sys
import os

In [5]:
sys.path.append(os.path.join(os.path.dirname('__file__'), 'models'))
sys.path.append(os.path.join(os.path.dirname('__file__'), 'scripts'))

In [6]:
from models.person import Person, Sex, TypePerson
from models.department import Department
from models.commune import Commune
from models.section import Section
from models.community import Community

In [7]:
from scripts.utils import random_item, random_date, random_date_between, random_int, years_between

In [8]:
haitian_names = pd.read_csv("./inputs/haitian_names.csv", encoding="latin-1")
kobo_df = pd.read_csv("./inputs/kobo_local.csv", encoding="latin-1")

In [9]:
h_fnames = list(haitian_names['FirstName'])

In [10]:
h_mnames = list(haitian_names['MiddleName'])

In [11]:
h_lnames = list(haitian_names['LastName'])

In [12]:
def get_fullname():
    f_name = random_item(h_fnames)
    m_name = random_item(h_mnames)
    l_name = random_item(h_lnames).upper()
    name = ""
    if isinstance(m_name, float):
        name = (f_name + " " + l_name).strip()
    else:
        if m_name != 'nan':
            name = (f_name + " " + m_name + " " + l_name).strip()
        else:
            name = (f_name + " " + m_name + " " + l_name).strip()
    return name

In [13]:
def get_first_name(h_fnames, h_mnames):
    f_name = random_item(h_fnames)
    m_name = random_item(h_mnames)
    name = ""
    if isinstance(m_name, float):
        name = f_name
    else:
        if m_name != 'nan':
            name = (f_name + " " + m_name).strip()
        else:
            name = (f_name).strip()
    return name


def get_last_name(h_lnames):
    return random_item(h_lnames)

In [14]:
#print(get_fullname())

In [15]:
#kobo_df

In [16]:
#departments_with_communes = kobo_df.groupby(['Departements', 'Communes', ])['Section communale/Ville'].apply(lambda x: list(set(x))).reset_index()
# Convert to an array (list of dictionaries)
#result = departments_with_communes.to_dict(orient='records')

In [17]:
#communes_with_sections = []
#for depart in result:
#    communes_with_sections.append(
#        {'name': depart["Communes"], "Department": depart['Departements'], "sections": depart['Section communale/Ville'] }
#    )

In [18]:
#len(communes_with_sections )

In [19]:
#departments_with_communes = kobo_df.groupby(['Departements', 'Communes', 'Section communale/Ville'])['Communauté/Addresse'].apply(lambda x: list(set(x))).reset_index()
#result2 = departments_with_communes.to_dict(orient='records')

In [20]:
#kobo_df

In [21]:
#communes_with_sections_locals = []
#for depart in result2:
#    communes_with_sections_locals.append(
#        {'name': depart["Communes"], "Department": depart['Departements'], "sections": depart['Section communale/Ville'], 'communities':depart['Communauté/Addresse'] }
#    )

In [22]:
#communes_with_sections_locals

In [23]:
#kobo_df

In [24]:
departments_with_structure = (
    kobo_df.groupby(['Departements', 'Communes', 'Section communale/Ville', 'Communauté/Addresse'])['Coordonnée GPS']
    .first()
    .reset_index()
    .groupby(['Departements', 'Communes', 'Section communale/Ville'])
    .apply(lambda x: x[['Communauté/Addresse', 'Coordonnée GPS']].to_dict(orient='records'), include_groups=False)
    .reset_index(name='Communauté/Addresse')
    .groupby(['Departements', 'Communes'])
    .apply(lambda x: x[['Section communale/Ville', 'Communauté/Addresse']].to_dict(orient='records'), include_groups=False)
    .reset_index(name='Section communale/Ville')
    .groupby('Departements')
    .apply(lambda x: x[['Communes', 'Section communale/Ville']].to_dict(orient='records'), include_groups=False)
    .reset_index(name='Communes')
)
result4 = departments_with_structure.to_dict(orient='records')

In [25]:
#Department(id=1, name="papa")
#result4[0]['Communes']

In [26]:
#communities_list

In [27]:
def get_department(data: list) -> dict:
    return random_item(data)

def get_commune(data: list) -> dict:
    return random_item(data)

def get_section(data: list) -> dict:
    return random_item(data)

def get_community(data: list) -> dict:
    return random_item(data)    

def get_gps_location(community: dict) -> tuple:
    gps_location = community['Coordonnée GPS'] 
    if gps_location is None:
        return (0,0) 
    if gps_location == '':
        return (0,0)
    gps_parts = gps_location.split(" ")
    return (float(gps_parts[0]), float(gps_parts[1])) 

In [28]:
#result4[0]['Communes']
#result4[0]
department_list = []
commune_list = []
section_list = []
community_list =[]
i = 1
j = 1
k = 1
l=1
for dep in result4:
    department= Department(
        id=i,
        name = dep['Departements']
    )
    department_list.append(department)
    communes = dep['Communes']
    for com in communes:
        commune = Commune(
            id=j,
            name=com['Communes'],
            department_id=i
        )
        commune_list.append(commune)
        sections = com['Section communale/Ville']
        
        for sec in sections:
            section = Section(
                id=k,
                name=sec['Section communale/Ville'],
                commune_id=j
            )
            section_list.append(section)
            communities = sec['Communauté/Addresse']
            for comm in communities:
                gps = get_gps_location(comm)
                community = Community(
                    id=l,
                    name=comm['Communauté/Addresse'],
                    section_id=k,
                    gps_lat=gps[0],
                    gps_lng=gps[1]
                )
                community_list.append(community)
                l = l+1
            k = k + 1
        j = j + 1
    i = i + 1

In [29]:
departments_dicts = [depart.dict() for depart in department_list]
depart_df = pd.DataFrame(departments_dicts)

In [30]:
communes_dicts = [com.dict() for com in commune_list]
com_df = pd.DataFrame(communes_dicts)

In [31]:
section_dicts = [sec.dict() for sec in section_list]
sec_df = pd.DataFrame(section_dicts)

In [32]:
comm_dicts = [comm.dict() for comm in community_list]
comm_df = pd.DataFrame(comm_dicts)

In [33]:
depart_df.to_sql('departments', con=engine, if_exists='replace', index=False)
com_df.to_sql('communes', con=engine, if_exists='replace', index=False)
sec_df.to_sql('sections', con=engine, if_exists='replace', index=False)
comm_df.to_sql('communities', con=engine, if_exists='replace', index=False)

226

In [34]:
birth_date_start = datetime(1940, 1, 1)
birth_date_end = datetime(2009, 1, 1)

joined_start_date = datetime(2024, 1, 1)
joined_end_date = datetime(2027, 12, 31)

#random_date = random_date_between(birth_date_start, birth_date_end)
#random_date = random_date_between(joined_start_date, joined_end_date )
#print(random_date)

In [35]:
#department = get_department(result4)

In [36]:
#commune = get_commune(department["Communes"])

In [37]:
#section = get_section(commune['Section communale/Ville'])
#section

In [38]:
#community = get_community(section['Communauté/Addresse'])

In [39]:
#get_gps_location(community)

In [40]:
participants = []
if not test:
    sexes = [Sex.male, Sex.female]
    type_p = [TypePerson.type_1, 
              TypePerson.type_2, 
              TypePerson.type_3, 
              TypePerson.type_4, 
              TypePerson.type_5, 
              TypePerson.type_6]
    
    for i in range(30141):
        #department = get_department(result4)
        #commune = get_commune(department["Communes"])
        #section = get_section(commune['Section communale/Ville'])
        #community = get_community(section['Communauté/Addresse'])
        #gps = get_gps_location(community)
        community = random_item(community_list)
        
        person: Person = Person(
               first_name = get_first_name(h_fnames, h_mnames),
               last_name = get_last_name(h_lnames),
               sex=random_item(sexes).value,
               address=community.name,
               community_id = community.id,
               phone_1='33511661',
               birth_date = random_date_between(birth_date_start, birth_date_end),
               date_joined = random_date_between(joined_start_date, joined_end_date ),
               nb_goat = random_int(0, 25),
               nb_cattle= random_int(0, 10),
               nb_sheep = random_int(0, 25),
               nb_poultry = random_int(0, 100),
               gps_lat = community.gps_lat,
               gps_lng = community.gps_lng,
               type_person = random_item(type_p).value
        )
        participants.append(person)

In [41]:
participants
people_dicts = [person.dict() for person in participants]
df = pd.DataFrame(people_dicts)

In [42]:
#df.head()

In [43]:
today = datetime.today()

In [44]:
ages = [years_between(today.date(), d) for d in df['birth_date']]

In [45]:
df['age'] = ages

In [46]:
#df.head()

In [47]:
age_group = ['15-29' if age < 30 else '30+' for age in df['age']]

In [48]:
df['age_group'] = age_group

In [49]:
df.groupby('age_group').count()

,id,first_name,last_name,address,sex,birth_date,birth_year,age,type_person,type_id,...,phone_1,phone_2,nb_goat,nb_cattle,nb_sheep,nb_poultry,gps_lat,gps_lng,date_joined,community_id
age_group,,,,,,,,,,,,,,,,,,,,,
15-29,0,6222,6222,6222,6222,6222,0,6222,6222,0,...,6222,0,6222,6222,6222,6222,6222,6222,6222,6222
30+,0,23919,23919,23919,23919,23919,0,23919,23919,0,...,23919,0,23919,23919,23919,23919,23919,23919,23919,23919


In [50]:
#df.to_csv('./outputs/participants.csv')
df['id'] = range(1, len(df) + 1)

In [51]:
df.to_sql('participants', con=engine, if_exists='replace', index=False)

30141

In [52]:
df_from_sql = pd.read_sql('participants', con=engine)

df_from_sql.head()

,id,first_name,last_name,address,sex,birth_date,birth_year,age,type_person,type_id,...,phone_2,nb_goat,nb_cattle,nb_sheep,nb_poultry,gps_lat,gps_lng,date_joined,community_id,age_group
0,1,Lysius,DEMOSTHÈNE,Lagodré,Male,1944-07-25,None,80,Personnes travaillant dans des entreprises du ...,None,...,None,11,7,13,20,18.185586,-73.787499,2027-10-17,176,30+
1,2,Rulx,LUCAS,Nan Pi a,Female,1959-11-25,None,64,Les gens au gouvernement,None,...,None,7,8,11,91,19.597446,-71.835456,2025-09-07,116,30+
2,3,Oreste,LAFAILLE,Galois,Female,1983-09-24,None,40,Personnes travaillant dans des entreprises du ...,None,...,None,3,1,24,84,19.660918,-72.269583,2027-07-05,70,30+
3,4,Alexis Dessalines,JACQUET,"Bois caïman,villa pestalozzi",Male,2005-07-04,None,19,Producteur non-petit exploitant,None,...,None,20,1,16,54,19.720357,-72.267424,2025-03-25,59,15-29
4,5,Charles Antoine,NELSON,Salmory,Female,1946-07-03,None,78,Personnes travaillant dans des entreprises du ...,None,...,None,3,8,15,12,19.118040,-72.041799,2025-04-15,29,30+


In [53]:
#from sqlalchemy import text

# Use `text()` to execute raw SQL queries
#with engine.connect() as conn:
#    result = conn.execute(text("SELECT * FROM participants WHERE age > 30"))
#    for row in result:
#        print(row)

In [54]:
#df_query = pd.read_sql('SELECT * FROM participants WHERE age > 30', con=engine)

#df_query.head()